<a href="https://colab.research.google.com/github/sabhinav3/Intro_to_ML/blob/main/abhi_assignment_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv("/content/Housing.csv")

In [3]:
data.shape

(545, 13)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [5]:
data.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [6]:
# dropping the columns that are not required for the model
columns_to_drop = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea", "furnishingstatus"]
data.drop(columns=columns_to_drop, inplace=True)
data

,price,area,bedrooms,bathrooms,stories,parking
0,13300000,7420,4,2,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,3,2,2,2
3,12215000,7500,4,2,2,3
4,11410000,7420,4,1,2,2
...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2
541,1767150,2400,3,1,1,0
542,1750000,3620,2,1,1,0
543,1750000,2910,3,1,1,0


In [10]:
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Assuming 'data' is your pandas DataFrame and 'price' is the column you want to reshape
data['price'] = data['price'].values.reshape(-1, 1)

X = data[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']]
y = data['price']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)  # Use .values to get a NumPy array
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values.reshape(-1, 1), dtype=torch.float32)  # Use .values to get a NumPy array


In [11]:
import torch
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        out = self.linear(x)
        return out

# Assuming input_dim is the number of features in your dataset
input_dim = X_train_tensor.shape[1]
model = LinearRegressionModel(input_dim)

In [12]:
# Training the model
def train_model(optimizer_name, learning_rate):
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'ADAM':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.MSELoss()

    # Training loop
    for epoch in range(5001):
        # Convert arrays to PyTorch variables
        inputs = Variable(X_train_tensor)
        targets = Variable(y_train_tensor)

        # Forward pass: Compute predicted y by passing x to the model
        outputs = model(inputs)

        # Compute and print loss
        loss = criterion(outputs, targets)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validation accuracy
        with torch.no_grad():  # We don't need gradients for validation
            model.eval()  # Set the model to evaluation mode
            y_pred = model(Variable(X_val_tensor))
            val_loss = criterion(y_pred, Variable(y_val_tensor)).item()
            r2_val = r2_score(y_val_tensor.numpy(), y_pred)
            model.train()  # Set the model back to training mode

        # Print stats every 500 epochs
        if epoch % 500 == 0:
            print(f'Optimizer: {optimizer_name}, Learning Rate: {learning_rate}, Epoch [{epoch}/5000], Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, R² Validation: {r2_val:.4f}')

    return loss.item(), val_loss, r2_val

# Train and validate the model with different optimizers and learning rates
learning_rates = [0.1, 0.01, 0.001, 0.0001]
optimizers = ['SGD', 'ADAM']
results = {}

for optimizer_name in optimizers:
    for lr in learning_rates:
        print(f'\nTraining with {optimizer_name} optimizer and learning rate {lr}')
        model = LinearRegressionModel(input_dim)  # Reset model for each training
        train_loss, val_loss, r2_val = train_model(optimizer_name, lr)
        results[(optimizer_name, lr)] = (train_loss, val_loss, r2_val)



Training with SGD optimizer and learning rate 0.1
Optimizer: SGD, Learning Rate: 0.1, Epoch [0/5000], Loss: 25234788843520.0000, Validation Loss: 19594865541120.0000, R² Validation: -2.8767
Optimizer: SGD, Learning Rate: 0.1, Epoch [500/5000], Loss: 1350008176640.0000, Validation Loss: 2292721647616.0000, R² Validation: 0.5464
Optimizer: SGD, Learning Rate: 0.1, Epoch [1000/5000], Loss: 1350008176640.0000, Validation Loss: 2292721647616.0000, R² Validation: 0.5464
Optimizer: SGD, Learning Rate: 0.1, Epoch [1500/5000], Loss: 1350008176640.0000, Validation Loss: 2292721647616.0000, R² Validation: 0.5464
Optimizer: SGD, Learning Rate: 0.1, Epoch [2000/5000], Loss: 1350008176640.0000, Validation Loss: 2292721647616.0000, R² Validation: 0.5464
Optimizer: SGD, Learning Rate: 0.1, Epoch [2500/5000], Loss: 1350008176640.0000, Validation Loss: 2292721647616.0000, R² Validation: 0.5464
Optimizer: SGD, Learning Rate: 0.1, Epoch [3000/5000], Loss: 1350008176640.0000, Validation Loss: 229272164761

In [13]:
# Report and select the best model based on minimum validation loss
best_setup = min(results, key=lambda k: results[k][1]) if results else None
2
if best_setup:
    best_train_loss, best_val_loss, best_r2_val = results[best_setup]

    print("\nBest model setup:", best_setup)
    print(f"Training Loss: {best_train_loss:.4f}, Validation Loss: {best_val_loss:.4f}")
    print(f"R² (R-squared) Validation: {best_r2_val:.4f} (higher is better)")
else:
    print("No results to report.")



Best model setup: ('SGD', 0.1)
Training Loss: 1350008176640.0000, Validation Loss: 2292721647616.0000
R² (R-squared) Validation: 0.5464 (higher is better)
